In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import random
import numpy as np


import math
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [2]:
import argparse
import json
from pathlib import Path
import random
import os
import schedulefree

import numpy as np
import torch
import wandb

import config
from data.utils import DataReader, get_dataset
import distributed
from models.utils import get_model
from optim.base import train
from optim.utils import cos_inf_schedule, wsd_schedule, get_batch

import sys
if 'ipykernel_launcher' in sys.argv[0]:
   sys.argv = sys.argv[:1]

def get_args():
    parser = argparse.ArgumentParser(allow_abbrev=False)
    parser.add_argument(
        "--config_format", default="base", choices=config.registered_formats()
    )
    args, rem_args = parser.parse_known_args()
    args.n_layer=1
    args.n_head=1
    args.n_embd=20
    args.vocab_size=20
    args.batch_size = 1
    args.multiple_of = 1
    args.dtype = "float32"
    args.datasets_dir = "/chenyupeng/data_files/llm_datasets"
    return config.parse_args_with_format(
        format=args.config_format, base_parser=parser, args=rem_args, namespace=args
    )

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
args = get_args()

import copy
def get_data_readers(args, verbose=True):
    data_srcs = get_dataset(args)
    train_reader = DataReader(
        data_src=data_srcs["train"],
        batch_size=args.batch_size,
        sequence_length=args.sequence_length,
        seed=args.data_seed,
        with_replacement=False,
        auto_shard=True,
        keep_in_ram=args.data_in_ram,
    )
    val_reader = DataReader(
        data_src=data_srcs["val"],
        batch_size=args.batch_size,
        sequence_length=args.sequence_length,
        seed=args.data_seed,
        with_replacement=False,
        auto_shard=False,  # NOTE Identical Per Rank
        keep_in_ram=args.data_in_ram,
    )

    if verbose:
        print(f"Num training tokens: {train_reader.num_tokens}")
        print(f"Num validation tokens: {val_reader.num_tokens}")

    return {
        "train": train_reader,
        "val": val_reader,
    }
data = get_data_readers(args)


model = get_model(args)

/chenyupeng/data_files/llm_datasets/slimpajama6B/
Num training tokens: 5827933038
Num validation tokens: 9479563


In [4]:
val_batches = []
data_reader = get_data_readers(args)["val"]
for _ in range(10):
    z_x, z_y = get_batch(data_reader, device="cuda")
    val_batches.append((z_x, z_y))
eval_batches = val_batches[0]  # 使用前10个batch评估

set_seed(100)
for i in range(eval_batches[0].shape[1]):
    eval_batches[0][0,i].data.copy_(random.randint(0, 19))
    if i>=1:
        eval_batches[1][0,i] = eval_batches[0][0,i-1]
eval_batches[1][0,-1] = random.randint(0, 19)
def compute_grad(model,eval_batches):
    model.train()
    total_loss = 0
    n_batches = 0
    # 清空梯度
    for p in model.parameters():
        p.grad = None
    
    # 梯度累积
    #for x, y in eval_batches:
    x = eval_batches[0]
    y = eval_batches[1]
    outputs = model(x, targets=y, get_logits=True)
    batch_loss = outputs["loss"]*1e4
    
    # 通过缩放损失实现梯度累积，相当于平均梯度
    batch_loss.backward()  # 梯度会累积

/chenyupeng/data_files/llm_datasets/slimpajama6B/
Num training tokens: 5827933038
Num validation tokens: 9479563


In [5]:
import torch.nn.functional as F
def get_hessian(model,eval_batches,a,r):
    compute_grad(model,eval_batches)
    grad_original = model.transformer.h[-1].mlp.c_proj.weight.grad.detach().clone()
    original_weight = model.transformer.h[-1].mlp.c_proj.weight.data.detach().clone()
    set_seed(42)
    random_phi = torch.randn_like(model.transformer.h[-1].mlp.c_proj.weight)
    for i in range(2000):
        #random_phi = random_phi/torch.norm(random_phi)
        model.transformer.h[-1].mlp.c_proj.weight.data.add_((random_phi/torch.norm(random_phi))*a)
        compute_grad(model,eval_batches)
        grad_after_pertu = model.transformer.h[-1].mlp.c_proj.weight.grad.data.detach().clone()
        random_phi = (1-r)*random_phi + (r/a)*(grad_after_pertu-grad_original)
        model.transformer.h[-1].mlp.c_proj.weight.data.copy_(original_weight)
        weight_norm_of_random = random_phi.norm()
        simi = F.cosine_similarity(grad_original.reshape(-1), (random_phi/torch.norm(random_phi)).reshape(-1), dim=0)
        print(f"{i}-th iteration, grad norm of phi: {weight_norm_of_random}, simi : {simi}")

    random_phi = random_phi/random_phi.norm()
    #cosine_simi = F.cosine_similarity(grad_original.reshape(-1), random_phi.reshape(-1), dim=0)
    return random_phi,simi

In [6]:
model = model.cuda()
model.eval()
phi,simi = get_hessian(model,eval_batches,0.1,0.1)

0-th iteration, grad norm of phi: 30.44900894165039, simi : -0.03006400354206562
1-th iteration, grad norm of phi: 27.370630264282227, simi : -0.02920096553862095
2-th iteration, grad norm of phi: 24.601749420166016, simi : -0.028230439871549606
3-th iteration, grad norm of phi: 22.111860275268555, simi : -0.02715757116675377
4-th iteration, grad norm of phi: 19.872901916503906, simi : -0.02591777965426445
5-th iteration, grad norm of phi: 17.859926223754883, simi : -0.024615805596113205
6-th iteration, grad norm of phi: 16.05162239074707, simi : -0.023162610828876495
7-th iteration, grad norm of phi: 14.426994323730469, simi : -0.02163754403591156
8-th iteration, grad norm of phi: 12.967682838439941, simi : -0.019925720989704132
9-th iteration, grad norm of phi: 11.65764045715332, simi : -0.01797616481781006
10-th iteration, grad norm of phi: 10.482236862182617, simi : -0.015725160017609596
11-th iteration, grad norm of phi: 9.429473876953125, simi : -0.013257702812552452
12-th iterat

In [11]:
phi.shape

torch.Size([20, 53])

In [60]:
model.transformer.h[-1].mlp.c_proj.weight.shape

torch.Size([20, 53])

In [31]:
model.transformer.h[-1].mlp.c_proj.weight

Parameter containing:
tensor([[ 0.0006, -0.0034,  0.0140,  ...,  0.0186, -0.0074,  0.0134],
        [-0.0225,  0.0089, -0.0028,  ..., -0.0131,  0.0037, -0.0078],
        [ 0.0032,  0.0076, -0.0307,  ...,  0.0057,  0.0003, -0.0009],
        ...,
        [ 0.0147, -0.0062, -0.0018,  ...,  0.0036,  0.0254, -0.0131],
        [-0.0121,  0.0002,  0.0063,  ...,  0.0144,  0.0037,  0.0080],
        [-0.0209,  0.0114,  0.0203,  ..., -0.0019,  0.0037, -0.0361]],
       device='cuda:0', requires_grad=True)

In [7]:
for p in model.parameters():
    p.grad = None

In [8]:
model = model.cuda()
model.zero_grad(set_to_none = True)
model.eval()
x = eval_batches[0]
y = eval_batches[1]
outputs = model(x, targets=y, get_logits=True)
batch_loss = outputs["loss"]*1e4

In [9]:
parameters = [p for n,p in model.named_parameters() if "mlp.c_proj" in n]

In [10]:
first_order_grads = torch.autograd.grad(batch_loss, parameters, create_graph=True,retain_graph=True)

In [11]:
hessian = torch.zeros((parameters[0].numel(), parameters[0].numel()), device=parameters[0].device)

In [12]:
#first_order_grads = first_order_grads[0].view(-1)
#hessian_vector = torch.autograd.grad(first_order_grads, parameters, retain_graph=True)


for i in range(parameters[0].numel()):
    grad2 = torch.autograd.grad(first_order_grads[0].flatten()[i].double(), parameters, retain_graph=True)[0]
            
    if grad2 is not None:
        hessian[i, :] = grad2.flatten()
    else:
        print("none")

In [14]:
u,v,d = torch.linalg.svd(torch.tensor(hessian).cuda().double())

/tmp/ipykernel_370/2357618817.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u,v,d = torch.linalg.svd(torch.tensor(hessian).cuda().double())


In [16]:
v

tensor([8.9354e+00, 7.7223e+00, 7.5865e+00,  ..., 9.2717e-07, 5.1113e-07,
        6.7599e-08], device='cuda:0', dtype=torch.float64)

In [17]:
torch.matmul(phi.reshape(1,-1), torch.matmul(hessian, phi.reshape(-1)))

tensor([8.9195], device='cuda:0')

In [8]:
model.zero_grad(set_to_none = True)
x = eval_batches[0]
y = eval_batches[1]
model.eval()
import time
def hessian_calculation(g_tensor, params):
    g_tensor = g_tensor.cuda()
    total_params = g_tensor.size(0)
    hessian_list = []
    t_d = time.time()
    for d in range(total_params):
        unit_vector = torch.zeros(total_params)
        unit_vector[d] = 1
        unit_vector = unit_vector.cuda()
        l = torch.sum(g_tensor * unit_vector)
        l.backward(retain_graph= True)
        hessian_row = []
        #print('name',name, param.grad)
        hessian_row.append(params[0].grad.double().data.clone())
        
        model.zero_grad(set_to_none = True)
        hessian_row = [g.flatten() for g in hessian_row] 
        hessian_row = [g.cpu() for g in hessian_row]
        hessian_row = torch.cat(hessian_row)
        #print('hessian_row', hessian_row)   
        hessian_list.append(hessian_row)
        # if d % 1000 == 0:
        #     print(f'Computing hessian: current batch = {batch_idx}/{self.num_batches}, current row of a hessian: {d}/{total_params}, total time = {time.time()- t_d} ')
    hessian = torch.stack(hessian_list, dim = 1)
    #print('hessian', hessian)   
    return hessian
full_hessian = 0
outputs = model(x, targets=y, get_logits=True)
batch_loss = outputs["loss"]*1e4
batch_loss.backward(create_graph= True)
g_list = []
count = 0
parameters = [p for n,p in model.named_parameters() if "mlp.c_proj" in n]
if parameters[0].requires_grad:
    count += parameters[0].numel()
    #print('g shape', param.grad , param.grad.shape)
    g_list.append(torch.flatten(parameters[0].grad.double()))
    #print('name',name, g_list[-1].size())
g_tensor = torch.cat(g_list, dim = 0)
#print('g_tensor',g_tensor)
model.zero_grad(set_to_none = True)
H = hessian_calculation(g_tensor,parameters)
full_hessian += H
full_hessian = torch.nan_to_num(full_hessian, nan = 0, posinf = 0, neginf = 0 )  # change nan, postive inf , negative inf, to 0
t_svd = time.time()
#print('doing EVD')
# _, eigenvalues, _ = torch.linalg.svd(full_hessian)  # ascending
#eigenvalues, _  = torch.eig(full_hessian)
full_hessian = full_hessian.numpy().astype(np.float64)
full_hessian = (full_hessian + full_hessian.T)/2 # make symetric, to 



#avoid numerical issue
#full_hessian = full_hessian.cuda()
#eigenvalues, _  = torch.linalg.eig(full_hessian)
# eigenvalues, _  = np.linalg.eigh(full_hessian)
# #_, eigenvalues, _ = np.linalg.svd(full_hessian) 
# eigenvalues = [eigen.item().real for eigen in eigenvalues]
# file_name = self.file_dir + 'eigenvalues.txt'
# with open(file_name, "w") as file:
#     for item in eigenvalues:
#         file.write(str(item)+"\n")

/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1177.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [21]:
full_hessian

(1060, 1060)

In [9]:
u,v,d = torch.linalg.svd(torch.tensor(full_hessian).cuda((.double())

In [10]:
v

tensor([1.1715e+01, 9.1922e+00, 8.8347e+00,  ..., 6.7539e-07, 5.1691e-07,
        9.5747e-08], dtype=torch.float64)

In [11]:
F.cosine_similarity(phi.reshape(-1), d[0,:],dim=0)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [23]:
u,v,d = torch.linalg.svd(hessian.double())

AttributeError: 'function' object has no attribute 'double'

In [65]:
v

tensor([8.4398e+00, 7.2601e+00, 6.8293e+00,  ..., 4.8793e-07, 2.4023e-07,
        2.1420e-07], device='cuda:0', dtype=torch.float64)

In [102]:
torch.matmul(hessian, phi.reshape(-1))

tensor([-2.8252e-07,  3.4268e-06,  1.2450e-04,  ...,  5.5907e-05,
        -1.9799e-05,  4.9336e-05], device='cuda:0')

In [54]:
(torch.matmul(hessian, phi.reshape(-1))).norm()

tensor(7.2583, device='cuda:0')

In [55]:
(phi.reshape(-1)*7.122495174407959 - torch.matmul(hessian, phi.reshape(-1))).norm()

tensor(0.2283, device='cuda:0')

In [56]:
F.cosine_similarity(phi.reshape(-1), d[0,:],dim=0)

tensor(0.0005, device='cuda:0', dtype=torch.float64)

In [26]:
torch.matmul(hessian, phi.reshape(-1))

tensor([-3.2368e-07,  5.7056e-06,  1.2471e-05,  ...,  3.9668e-07,
         2.2853e-07, -1.4662e-06], device='cuda:0')

In [28]:
phi.reshape(-1)*8e-4

tensor([-1.6113e-05,  3.2472e-06,  1.0434e-05,  ..., -5.9264e-07,
         4.9677e-07, -2.0703e-06], device='cuda:0')

In [122]:
phi[:2,:10]

tensor([[-0.0012,  0.0051,  0.1549,  0.0139, -0.0057,  0.0166, -0.0391, -0.0140,
          0.0126,  0.0058],
        [ 0.0037, -0.0004, -0.0201, -0.0034, -0.0018,  0.0006, -0.0068, -0.0008,
         -0.0059,  0.0016]], device='cuda:0')

In [126]:
v[:7]

tensor([0.0011, 0.0010, 0.0009, 0.0008, 0.0008, 0.0008, 0.0007],
       device='cuda:0', dtype=torch.float64)